In [1]:
import os
from datetime import datetime

#PROJECT_ROOT_DIR = "/home/ubuntu/TroubledLife"
PROJECT_ROOT_DIR = "/Users/gopora/MyStuff/Dev/Workspaces/Sandbox/TroubledLife"
DATASETS_DIR = os.path.join(PROJECT_ROOT_DIR, "data")
TF_LOG_DIR = os.path.join(PROJECT_ROOT_DIR, "tf_logs")
MODEL_CHECKPOINTS_DIR = os.path.join(PROJECT_ROOT_DIR, "model_checkpoints")
TRAINING_SET_DATA_FILE = "troubled_life_policy_train_data.csv"
TEST_SET_DATA_FILE = "troubled_life_policy_test_data.csv"

now = datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = "{}/run-{}/".format(TF_LOG_DIR, now)


import data_preparation as dp

runtime = 5

#dp.generate_troubled_life_policy_data(no_of_policies=10000, runtime=5, file_path=os.path.join(DATASETS_DIR, TRAINING_SET_DATA_FILE))

#dp.generate_troubled_life_policy_data(no_of_policies=2000, runtime=5, file_path=os.path.join(DATASETS_DIR, TEST_SET_DATA_FILE))

policy_histories_train = \
    dp.load_troubled_life_policy_data(file_path=os.path.join(DATASETS_DIR, TRAINING_SET_DATA_FILE))

policy_histories_test = \
    dp.load_troubled_life_policy_data(file_path=os.path.join(DATASETS_DIR, TEST_SET_DATA_FILE))

policy_histories_length_train, max_policy_history_length_train = \
    dp.get_policy_history_lengths(policy_histories=policy_histories_train)

policy_histories_length_test, max_policy_history_length_test = \
    dp.get_policy_history_lengths(policy_histories=policy_histories_test)

max_length_policy_history = max(max_policy_history_length_train, max_policy_history_length_test)



# Pad the histories up to maximum length of both, train and test set

# policy_histories_train = \
#     dp.pad_troubled_life_policy_histories(policy_histories=policy_histories_train,
#                                           policy_histories_lengths=policy_histories_length_train,
#                                           max_policy_history_length=max_policy_history_length)
# 
# policy_histories_test = \
#     dp.pad_troubled_life_policy_histories(policy_histories=policy_histories_test,
#                                           policy_histories_lengths=policy_histories_length_test,
#                                           max_policy_history_length=max_policy_history_length)
# 
# # # Save padded data, since always generating and padding takes too long
# policy_histories_train.to_csv(path_or_buf=os.path.join(DATASETS_DIR, TRAINING_SET_DATA_FILE))
# policy_histories_test.to_csv(path_or_buf=os.path.join(DATASETS_DIR, TEST_SET_DATA_FILE))

# Extract features and labels from dataset as numpy.ndarray(s)
binary_classification = True

train_labels, train_features, train_seq_lengths =\
    dp.prepare_labels_features_lengths(policy_histories=policy_histories_train,
                                       policy_histories_lengths=policy_histories_length_train, 
                                       max_policy_history_length=max_length_policy_history,
                                       binary_classification=binary_classification)
test_labels, test_features, test_seq_lengths =\
    dp.prepare_labels_features_lengths(policy_histories=policy_histories_test,
                                       policy_histories_lengths=policy_histories_length_test, 
                                       max_policy_history_length=max_length_policy_history,
                                       binary_classification=binary_classification)

train_data = dp.TrainDataSet(train_labels=train_labels, train_features=train_features, train_seq_lengths=train_seq_lengths)

In [2]:
def generator_loss(size_batch, Gz, Dg, init_z):
    # discriminator should ideally fall for the fakes, so the discriminator's logits should flag 0
    zeros = tf.zeros(size_batch, tf.int32)

    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=zeros, logits=Dg)
    loss = tf.reduce_mean(loss)

    accuracy = tf.reduce_mean(tf.cast(tf.nn.in_top_k(Dg, zeros, 1), tf.float32))
    
    interest = init_z[:, 1]
    premium = init_z[:, 0]
    
    k1 = premium
    k1v = k1 * (1 + interest)
    k2 = premium + k1v
    k2v = k2 * (1 + interest)
    k3 = premium + k2v
    k3v = k3 * (1 + interest)
    k4 = premium + k3v
    k4v = k4 * (1 + interest)
    k5 = premium + k4v
    k5v = k5 * (1 + interest)
    
    loss2 = tf.reduce_mean(tf.abs(Gz[:, 0, 0] - premium) + tf.abs(Gz[:, 0, 1]) +
                           tf.abs(Gz[:, 1, 0] - premium) + tf.abs(Gz[:, 1, 1] - k1) +
                           tf.abs(Gz[:, 2, 0] - premium) + tf.abs(Gz[:, 2, 1] - k1v) +
                           tf.abs(Gz[:, 3, 0] - premium) + tf.abs(Gz[:, 3, 1] - k2) +
                           tf.abs(Gz[:, 4, 0] - premium) + tf.abs(Gz[:, 4, 1] - k2v) +
                           tf.abs(Gz[:, 5, 0] - premium) + tf.abs(Gz[:, 5, 1] - k3) +
                           tf.abs(Gz[:, 6, 0] - premium) + tf.abs(Gz[:, 6, 1] - k3v) +
                           tf.abs(Gz[:, 7, 0] - premium) + tf.abs(Gz[:, 7, 1] - k4) +
                           tf.abs(Gz[:, 8, 0] - premium) + tf.abs(Gz[:, 8, 1] - k4v) +
                           tf.abs(Gz[:, 9, 0] - premium) + tf.abs(Gz[:, 9, 1] - k5) +
                           tf.abs(Gz[:, 10, 0] - premium) + tf.abs(Gz[:, 10, 1] - k5v)
                           )

    return loss2, accuracy

In [ ]:
import tensorflow as tf
import gan as gan


size_batch = 200

# Generator hyperparameters
gru_g=False
leaky_g=False
n_inputs_g = 2
n_layers_g = 1
n_neurons_g = 20
n_outputs_g = 2
input_keep_prob_g = 1.0
learning_rate_g = 1e-2
beta1_g=0.99
beta2_g=0.999
epsilon_g=1e-08
clip_g=False
threshold_g = 1.0

# Discriminator hyperparameters
learning_rate_d = 0.0001
n_inputs_d = 2
n_layers_d = 3
n_neurons_d = 200
n_outputs_d = 2 if binary_classification else max_length_policy_history


tf.reset_default_graph()
tf.set_random_seed(42)

Z = tf.placeholder(tf.float32, [None, max_length_policy_history, n_inputs_g], name='Z')
init_z = tf.placeholder(tf.float32, [None, n_inputs_g], name='init_z')
seq_length_z = tf.placeholder(tf.int32, [None], name='seq_length_z')

X = tf.placeholder(tf.float32, [None, max_length_policy_history, n_inputs_d], name="X")
y = tf.placeholder(tf.int32, [None], name="y")
seq_length_x = tf.placeholder(tf.int32, [None], name="seq_length_x")

Gz = gan.generator(Z=Z, n_outputs=n_outputs_g, n_layers=n_layers_g, n_neurons=n_neurons_g, 
                   seq_length=seq_length_z, gru=gru_g, leaky=leaky_g, input_keep_prob=input_keep_prob_g)

Dx, y_pred_x = gan.discriminator(X=X, seq_length=seq_length_x, 
                                 n_layers=n_layers_d, n_neurons=n_neurons_d, n_outputs=n_outputs_d, leaky=True)
Dg, y_pred_g = gan.discriminator(X=Gz, seq_length=seq_length_z,
                                 n_layers=n_layers_d, n_neurons=n_neurons_d, n_outputs=n_outputs_d, leaky=True, reuse=True)

loss_g, accuracy_g = generator_loss(size_batch=size_batch, Gz=Gz, Dg=Dg, init_z=init_z)
loss_real_d, accuracy_real_d = gan.discriminator_loss_real(Dx=Dx, y=y)
loss_fake_d, accuracy_fake_d = gan.discriminator_loss_fake(size_batch=size_batch, Dg=Dg)

g_trainer = gan.generator_trainer(learning_rate=learning_rate_g, beta1=beta1_g, beta2=beta2_g, epsilon=epsilon_g, 
                                  clip=clip_g, threshold=threshold_g, loss=loss_g)
d_trainer = gan.discriminator_trainer_real(learning_rate_d, loss_real_d + loss_fake_d)

tf.summary.scalar('Generator_loss', loss_g)
#tf.summary.scalar('Discriminator_loss_real', loss_real_d)
#tf.summary.scalar('Discriminator_loss_fake', loss_fake_d)

merged = tf.summary.merge_all()
writer = tf.summary.FileWriter(log_dir, tf.get_default_graph())

tvars = tf.trainable_variables()

g_saver = tf.train.Saver(var_list=[var for var in tvars if "t_generator" in var.name])
d_saver = tf.train.Saver(var_list=[var for var in tvars if "t_discriminator" in var.name])


NameError: name 'gru_g' is not defined

In [4]:
import numpy as np

np.set_printoptions(formatter={'float_kind': (lambda x: "%.2f" % x)})

sess = tf.Session()

sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())

In [ ]:
n_epochs = 5

# Pre-train discriminator
for epoch in range(n_epochs):
    for batch in range(train_data.num_examples // size_batch):
        Z_batch, seq_length_z_batch, _ = dp.generate_Z_batch(
            size_batch=size_batch, max_length_policy_history=max_length_policy_history, n_inputs = n_inputs_g, runtime=runtime)
 
        y_batch, X_batch, seq_length_x_batch = train_data.next_batch(size_batch)

        _, lossRealD, lossFakeD, accRealD, accFakeD = \
            sess.run([d_trainer, loss_real_d, loss_fake_d, accuracy_real_d, accuracy_fake_d],
                     {X: X_batch, y: y_batch, seq_length_x: seq_length_x_batch,
                      Z: Z_batch, seq_length_z: seq_length_z_batch})

    print("Epoch:", epoch, "lossRealD:", lossRealD, "accRealD:", accRealD,
          "lossFakeD:", lossFakeD, "accFakeD:", accFakeD)

d_saver.save(sess, os.path.join(MODEL_CHECKPOINTS_DIR, "discriminator.ckpt"))

In [ ]:
#d_saver.restore(sess, os.path.join(MODEL_CHECKPOINTS_DIR, "discriminator.ckpt"))
#g_saver.restore(sess, os.path.join(MODEL_CHECKPOINTS_DIR, "generator.ckpt"))

# Train generator and discriminator together
n_epochs = 200

for epoch in range(n_epochs):
    for batch in range(train_data.num_examples // size_batch):
        Z_batch, seq_length_z_batch, init_z_batch = dp.generate_Z_batch(
            size_batch=size_batch, max_length_policy_history=max_length_policy_history, 
            n_inputs = n_inputs_g, runtime=runtime)
        
        # y_batch, X_batch, seq_length_x_batch = train_data.next_batch(size_batch)
        # 
       #   # Train discriminator on both real and fake data
       #  _, lossRealD, accRealD = \
       #      sess.run([d_trainer, loss_real_d, accuracy_real_d],
       #               {X: X_batch, y: y_batch, seq_length_x: seq_length_x_batch,
       #                Z: Z_batch, seq_length_z: seq_length_z_batch})
       # 
       #  Z_batch, seq_length_z_batch, init_z_batch = dp.generate_Z_batch(
       #      size_batch=size_batch, max_length_policy_history=max_length_policy_history, 
       #      n_inputs = n_inputs_g, runtime=runtime)

        # # Train generator    
        # _, lossFakeD, accFakeD, lossG, accG, gData = \
        #     sess.run([g_trainer, loss_fake_d, accuracy_fake_d, loss_g, accuracy_g, Gz], 
        #              feed_dict={Z: Z_batch, seq_length_z: seq_length_z_batch, init_z: init_z_batch})

        # Train generator    
        _, lossG, gData = \
            sess.run([g_trainer, loss_g, Gz], 
                     feed_dict={Z: Z_batch, seq_length_z: seq_length_z_batch, init_z: init_z_batch})

    # summary = sess.run(merged, feed_dict={X: X_batch, y: y_batch, seq_length_x: seq_length_x_batch,
    #                       Z: Z_batch, seq_length_z: seq_length_z_batch, init_z: init_z_batch})
    # writer.add_summary(summary, epoch)

    summary = sess.run(merged, feed_dict={Z: Z_batch, seq_length_z: seq_length_z_batch, init_z: init_z_batch})
    writer.add_summary(summary, epoch)

    print("Epoch:", epoch, 
          #"lossRealD:", lossRealD, "accRealD:", accRealD,
          #"lossFakeD:", lossFakeD, "accFakeD:", accFakeD,
          #"lossG:", lossG, "accG:", accG)
          "lossG:", lossG)
    
    print(Z_batch[0, 0])
    print(gData[0])
    
g_saver.save(sess, os.path.join(MODEL_CHECKPOINTS_DIR, "generator3.ckpt"))    

Epoch: 0 lossG: 9240.36
[1250.00 0.03]
[[707.26 250.20]
 [715.16 1817.54]
 [549.20 2382.25]
 [693.70 2718.96]
 [673.04 3061.48]
 [906.20 3731.88]
 [967.64 4277.74]
 [1229.20 5113.80]
 [1338.84 5869.85]
 [1660.36 6960.55]
 [1841.75 8026.02]
 [0.13 0.28]
 [0.13 0.28]
 [0.13 0.28]
 [0.13 0.28]]
Epoch: 1 lossG: 7140.05
[1250.00 0.03]
[[1327.73 -75.97]
 [813.44 1825.00]
 [559.21 2008.93]
 [889.22 2663.53]
 [774.44 2860.46]
 [1140.17 3681.51]
 [1106.12 4017.47]
 [1493.75 4939.65]
 [1499.71 5430.55]
 [1964.08 6559.65]
 [2003.41 7249.20]
 [0.38 0.27]
 [0.38 0.27]
 [0.38 0.27]
 [0.38 0.27]]
Epoch: 2 lossG: 6283.93
[1250.00 0.05]
[[1399.29 25.45]
 [871.05 1865.88]
 [398.85 1878.41]
 [919.68 2831.36]
 [557.86 2804.06]
 [1088.77 3890.08]
 [817.43 3990.43]
 [1356.84 5139.58]
 [1113.48 5380.03]
 [1726.46 6713.05]
 [1480.61 7121.16]
 [0.64 0.24]
 [0.64 0.24]
 [0.64 0.24]
 [0.64 0.24]]
Epoch: 3 lossG: 5260.43
[1250.00 0.04]
[[1154.62 15.45]
 [965.91 1627.56]
 [398.24 1599.77]
 [1034.02 2735.85]
 [554.

Epoch: 28 lossG: 541.571
[1250.00 0.04]
[[1250.05 2.38]
 [1247.69 1247.63]
 [1250.71 1301.52]
 [1239.54 2543.71]
 [1250.02 2650.49]
 [1243.30 3904.48]
 [1249.61 4059.41]
 [1246.48 5310.85]
 [1251.97 5523.48]
 [1247.42 6770.06]
 [1253.79 7043.79]
 [2.17 -0.04]
 [2.17 -0.04]
 [2.17 -0.04]
 [2.17 -0.04]]
Epoch: 29 lossG: 604.071
[1250.00 0.04]
[[1251.60 -4.98]
 [1250.11 1249.41]
 [1241.67 1299.81]
 [1247.97 2550.76]
 [1241.75 2650.47]
 [1242.12 3904.28]
 [1236.42 4054.44]
 [1240.28 5304.05]
 [1236.69 5513.87]
 [1238.03 6758.42]
 [1238.34 7031.52]
 [2.17 -0.04]
 [2.17 -0.04]
 [2.17 -0.04]
 [2.17 -0.04]]
Epoch: 30 lossG: 659.477
[1250.00 0.03]
[[1253.76 1.28]
 [1257.42 1251.59]
 [1249.25 1307.42]
 [1248.83 2555.30]
 [1245.07 2662.10]
 [1244.41 3916.85]
 [1238.73 4075.59]
 [1241.84 5328.62]
 [1236.65 5548.99]
 [1238.18 6799.11]
 [1235.18 7084.94]
 [2.17 -0.04]
 [2.17 -0.04]
 [2.17 -0.04]
 [2.17 -0.04]]
Epoch: 31 lossG: 815.645
[1250.00 0.03]
[[1251.15 3.57]
 [1246.88 1248.49]
 [1243.73 1299.

In [10]:
g_saver.save(sess, os.path.join(MODEL_CHECKPOINTS_DIR, "generator.ckpt"))

'/home/ubuntu/TroubledLife/model_checkpoints/generator.ckpt'

In [4]:
for var in tvars:
    if "t_generator" in var.name:
        print(var) 

<tf.Variable 't_generator/rnn/multi_rnn_cell/cell_0/basic_rnn_cell/kernel:0' shape=(22, 20) dtype=float32_ref>
<tf.Variable 't_generator/rnn/multi_rnn_cell/cell_0/basic_rnn_cell/bias:0' shape=(20,) dtype=float32_ref>
<tf.Variable 't_generator/dense/kernel:0' shape=(20, 2) dtype=float32_ref>
<tf.Variable 't_generator/dense/bias:0' shape=(2,) dtype=float32_ref>
